In [131]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import numpy as np

In [133]:
train_datadir = 'cat-dog/cat-dog-train'
test_datadir = 'cat-dog/cat-dog-val'

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), # 随机水平翻转
    transforms.RandomRotation(15), # 随机旋转(+/- 15度)
    transforms.Resize([224, 224]), # 调整图像大小
    transforms.ToTensor(), # 转为张量
    transforms.Normalize( # 标准化(ImageNet参数)
        mean=[0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225])
])

# 测试数据集预处理
test_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = [0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225])
])

# 加载数据集
train_data = datasets.ImageFolder(train_datadir, transform=train_transforms)
test_data = datasets.ImageFolder(test_datadir, transform=test_transforms)

# 创建数据加载器
train_loader = torch.utils.data.DataLoader(train_data,
                                      batch_size=4,
                                      shuffle=True,
                                      num_workers=1)
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=1)

# 打印类别信息
print(train_loader.dataset.classes)
print(train_loader.dataset)

['cat', 'dog']
Dataset ImageFolder
    Number of datapoints: 480
    Root location: cat-dog/cat-dog-train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
               Resize(size=[224, 224], interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [135]:
for X, y in test_loader:
    print("Shape of X [N, C, H, W]:", X.shape)
    print("Shape of y:", y.shape, y.type)
    break

Shape of X [N, C, H, W]: torch.Size([4, 3, 224, 224])
Shape of y: torch.Size([4]) tensor([1, 1, 1, 1])


In [136]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# 定义模型
class NeutralNetwork(nn.Module):
    def __init__(self):
        super(NeutralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.Linear_relu_stack = nn.Sequential(
            nn.Linear(3*224*224, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.flatten(x) # 展平 图像
        logits = self.Linear_relu_stack(x)
        return logits

model = NeutralNetwork().to(device) # 自动选择CPU/GPU
print(model)

Using cpu device
NeutralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (Linear_relu_stack): Sequential(
    (0): Linear(in_features=150528, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=256, out_features=2, bias=True)
  )
)


In [137]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [141]:
def train(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = criterion(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), batch_idx*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [143]:
def test(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches # 计算平均损失和准确率
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [145]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------------")
    train(train_loader, model, criterion, optimizer)
    test(test_loader, model, criterion)

print("Done!")

Epoch 1
-----------------------------------
loss: 0.604318 [    0/  480]
loss: 139.880951 [  400/  480]
Test Error: 
 Accuracy: 59.2%, Avg loss: 36.504234 

Epoch 2
-----------------------------------
loss: 3.189614 [    0/  480]
loss: 31.315996 [  400/  480]
Test Error: 
 Accuracy: 60.8%, Avg loss: 35.214924 

Epoch 3
-----------------------------------
loss: 16.931362 [    0/  480]
loss: 5.122310 [  400/  480]
Test Error: 
 Accuracy: 74.2%, Avg loss: 5.363300 

Epoch 4
-----------------------------------
loss: 0.000028 [    0/  480]
loss: 9.088091 [  400/  480]
Test Error: 
 Accuracy: 79.2%, Avg loss: 2.232654 

Epoch 5
-----------------------------------
loss: 0.419220 [    0/  480]
loss: 1.802551 [  400/  480]
Test Error: 
 Accuracy: 69.2%, Avg loss: 3.148292 

Epoch 6
-----------------------------------
loss: 0.760963 [    0/  480]
loss: 1.797799 [  400/  480]
Test Error: 
 Accuracy: 77.5%, Avg loss: 1.387486 

Epoch 7
-----------------------------------
loss: 0.967791 [    0/  48